In [ ]:
!pip install google-generativeai python-docx python-dotenv
!pip install -U sentence-transformers

import os
import json
import docx
import time
import re
import google.generativeai as genai
from dotenv import load_dotenv
from google.api_core import exceptions

# --- 準備 ---
load_dotenv()
try:
    # APIキーを設定
    api_key = os.environ.get("GOOGLE_API_KEY")
    if not api_key:
        raise ValueError("APIキーが.envファイルに設定されていません。")
    genai.configure(api_key=api_key)
except ValueError as e:
    print(e)
    exit()

# --- 設定値 ---
MODEL_FOR_EMBEDDING = 'text-embedding-004'
CHUNK_SIZE = 2500 # バイト数上限エラーを避けるため、安全な値に設定

# --- 関数定義 ---

def get_text_from_docx(filepath):
    """docxファイルからテキストを抽出する"""
    try:
        doc = docx.Document(filepath)
        return '\n'.join([para.text for para in doc.paragraphs])
    except Exception as e:
        print(f"  -> エラー: {filepath} の読み込みに失敗 - {e}")
        return None

def extract_key_sections(text):
    """論文テキストから要旨、はじめに、おわりの部分を抽出する"""
    abstract_kw = r"要旨|要約|Abstract"
    intro_kw = r"はじめに|序論|緒言|Introduction"
    conclusion_kw = r"おわりに|結論|結言|Conclusion"
    
    sections = {"abstract": "", "intro": "", "conclusion": ""}
    reading_section = None
    
    for line in text.split('\n'):
        line_stripped = line.strip()
        # 正規表現のマッチングでセクションの開始を判定
        if re.match(f"^({abstract_kw})$", line_stripped, re.IGNORECASE): reading_section = "abstract"
        elif re.match(f"^({intro_kw})$", line_stripped, re.IGNORECASE): reading_section = "intro"
        elif re.match(f"^({conclusion_kw})$", line_stripped, re.IGNORECASE): reading_section = "conclusion"
        
        if reading_section:
            sections[reading_section] += line + "\n"
            
    return (sections["abstract"] + sections["intro"] + sections["conclusion"]).strip()

def create_embedding_with_fallback(text, title):
    """全文 → 主要セクション → チャンク化、という優先順位でEmbeddingを試行する。"""
    
    # 1. 全文で試行
    try:
        print("  -> 全文でのEmbedding生成を試行...")
        response = genai.embed_content(model=MODEL_FOR_EMBEDDING, content=text, task_type="RETRIEVAL_DOCUMENT", title=title)
        print("  -> 全文での生成に成功。")
        return response['embedding']
    except exceptions.InvalidArgument as e:
        if "payload size exceeds the limit" not in str(e):
            print(f"  -> 予期せぬエラー（サイズ上限以外）: {e}")
            return None

        # 2. サイズエラーなら、主要セクション抽出を試行
        print("  -> 全文が長すぎるため、主要セクションの抽出を試行...")
        key_text = extract_key_sections(text)
        
        content_to_embed = None
        source_description = ""

        if key_text:
            print("  -> 主要セクションを抽出。これでEmbeddingを生成します。")
            content_to_embed = key_text
            source_description = "主要セクション"
        else:
            print("  -> 主要セクションが見つからず。最終手段として「最初・真ん中・最後」を抽出します。")
            # 全文をチャンク化の対象とする
            content_to_embed = text
            source_description = "全文のチャンク"

        try:
            # 3. それでもテキストが長すぎる場合はチャンク化する
            if len(content_to_embed) > 15000: # 念のためここでも長さをチェック
                 print(f"  -> 「{source_description}」も長すぎるため、最終手段でチャンク化します。")
                 start_text = content_to_embed[:CHUNK_SIZE]
                 mid_point = len(content_to_embed) // 2
                 middle_text = content_to_embed[mid_point - (CHUNK_SIZE // 2) : mid_point + (CHUNK_SIZE // 2)]
                 end_text = content_to_embed[-CHUNK_SIZE:]
                 chunked_text = f"【論文の冒頭】\n{start_text}\n\n【論文の中間部分】\n{middle_text}\n\n【論文の末尾】\n{end_text}"
                 content_to_embed = chunked_text
                 source_description = "チャンク版"

            response = genai.embed_content(model=MODEL_FOR_EMBEDDING, content=content_to_embed, task_type="RETRIEVAL_DOCUMENT", title=title)
            print(f"  -> 「{source_description}」での生成に成功。")
            return response['embedding']
        except Exception as e_retry:
            print(f"  -> フォールバック処理中にエラーが発生: {e_retry}")
            return None
            
    except Exception as e:
        print(f"  -> 予期せぬエラーが発生しました: {e}")
        return None

# --- 実行部分 ---
papers_dir = './papers'
all_papers_data = []

if not os.path.isdir(papers_dir):
    print(f"エラー: '{papers_dir}' ディレクトリが見つかりません。")
else:
    print("論文の分析とEmbedding生成を開始します...")
    docx_files = [f for f in os.listdir(papers_dir) if f.endswith('.docx')]
    for docx_file in docx_files:
        filepath = os.path.join(papers_dir, docx_file)
        print(f"{docx_file}を処理中...")
        text = get_text_from_docx(filepath)
        if not text:
            continue

        embedding = create_embedding_with_fallback(text, docx_file)
        
        if embedding:
            all_papers_data.append({
                "filename": docx_file,
                "content": text[:500] + "...",
                "embedding": embedding
            })
        else:
            print(f"  -> {docx_file} のEmbedding生成に最終的に失敗しました。スキップします。")
        
        time.sleep(2) 

# --- 最終結果をJSONファイルに書き出す ---
with open('papers.json', 'w', encoding='utf-8') as f:
    json.dump(all_papers_data, f, ensure_ascii=False, indent=2)

print("\n全ての論文のEmbedding生成が完了し、'papers.json'に保存しました。")

C:\Users\kotas\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Embedding用AIモデルの読み込みを開始します...


C:\Users\kotas\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\kotas\.cache\huggingface\hub\models--intfloat--multilingual-e5-base. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


モデルの読み込み完了。

論文のEmbedding生成を開始します...
02_内山田湖太_卒論_最終版.docxを処理中...
  -> Embedding生成完了。
EI_16_田中慶人_卒論v4.docxを処理中...
  -> Embedding生成完了。
kaku.docxを処理中...
  -> Embedding生成完了。
kazuki.docxを処理中...
  -> Embedding生成完了。
mae.docxを処理中...
  -> Embedding生成完了。
naka.docxを処理中...
  -> Embedding生成完了。
oisi - コピー - コピー.docxを処理中...
  -> Embedding生成完了。
oisi - コピー.docxを処理中...
  -> Embedding生成完了。
oisi.docxを処理中...
  -> Embedding生成完了。
soma - コピー.docxを処理中...
  -> Embedding生成完了。
soma.docxを処理中...
  -> Embedding生成完了。
自分の卒論.docxを処理中...
  -> Embedding生成完了。

全ての論文のEmbedding生成が完了し、'papers.json'に保存しました。
